In [4]:
#astropy library
import astropy.coordinates as coord
from astropy.table import QTable
import astropy.units as u
from astroquery.gaia import Gaia

#third-party imports
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

#gala imports
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

#LATAR BELAKANG ILMIAH
Misi Gaia adalah misi ESA yang bertujuan untuk mengukur posisi 3D dan kecepatan sejumlah besar bintang di seluruh Bima Sakti. Tujuan misi utama adalah untuk memungkinkan mempelajari pembentukan, struktur, dan sejarah evolusi Galaksi kita dengan mengukur astrometri (posisi langit, paralaks, dan gerakan yang tepat) untuk sekitar 2 miliar bintang lebih terang dari Gaia G-band besarnya fotometrik G ≲ 21.Pada akhir misi (~ 2022), Gaia juga akan menyediakan fotometri multi-band dan spektrofotometri resolusi rendah untuk sumber-sumber ini, bersama dengan kecepatan radial atau line-of-sight untuk subsampel sekitar 100 juta bintang.

Pada April 2018, Gaia secara terbuka merilis katalog data utama pertamanya — rilis data 2 atau DR2 — yang menyediakan subset data ini kepada siapa pun yang memiliki koneksi internet. Dalam tutorial ini, kita akan menggunakan astrometri, kecepatan radial, dan fotometri untuk subset kecil DR2 untuk mempelajari kinematika berbagai jenis bintang di Bima Sakti.

In [7]:
query_text = '''SELECT TOP 4096 ra, dec, parallax, pmra, pmdec, radial_velocity,
phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag
FROM gaiadr2.gaia_source
WHERE parallax_over_error > 10 AND
    parallax > 10 AND
    radial_velocity IS NOT null
ORDER BY random_index
'''
job = Gaia.launch_job(query_text)
gaia_data = job.get_results()
gaia_data.write('gaia_data.fits')


In [8]:
gaia_data = QTable.read('gaia_data.fits')

In [9]:
gaia_data[:4]

ra,dec,parallax,pmra,pmdec,radial_velocity,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag
deg,deg,mas,mas / yr,mas / yr,km / s,mag,mag,mag
float64,float64,float64,float64,float64,float64,float32,float32,float32
256.8370985186671,46.5081010273448,10.770964834345387,-19.76657104695835,55.92605676622464,-17.673072044811907,12.956912994384766,14.192928314208984,11.859498023986816
129.45826809010867,-6.806694022293258,41.081670013380716,-297.34595349374183,43.83424795415093,35.53814452290586,6.576352119445801,6.931065082550049,6.111272811889648
2.317856723637118,9.00699655780345,17.824765284426462,-171.8278607514459,-1083.8740928197751,-148.91780163518774,13.119074821472168,13.968703269958496,12.223302841186523
330.41287472805976,51.88897801553169,13.0756133009657,-85.30745872006916,-34.23366534215265,-28.726469345049587,12.101919174194336,13.05264949798584,11.149785995483398


In [10]:
dist = coord.Distance(parallax=u.Quantity(gaia_data['parallax']))
dist.min(), dist.max()


(<Distance 8.79909291 pc>, <Distance 99.99985292 pc>)

In [11]:
c = coord.SkyCoord(ra=gaia_data['ra'], 
                   dec=gaia_data['dec'],
                   distance=dist,
                   pm_ra_cosdec=gaia_data['pmra'], 
                   pm_dec=gaia_data['pmdec'],
                   radial_velocity=gaia_data['radial_velocity'])

In [12]:
c[:4]

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(256.83709852, 46.50810103, 92.84219338),
     (129.45826809, -6.80669402, 24.3417563 ),
     (  2.31785672,  9.00699656, 56.1017205 ),
     (330.41287473, 51.88897802, 76.47824825)]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [( -19.76657105,    55.92605677,  -17.67307204),
     (-297.34595349,    43.83424795,   35.53814452),
     (-171.82786075, -1083.87409282, -148.91780164),
     ( -85.30745872,   -34.23366534,  -28.72646935)]>